In [10]:
import os
import pytz
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# from cacheops import invalidate_all
from django.apps import apps as django_apps
from flourish_caregiver.helper_classes.utils import cohort_assigned
from flourish_caregiver.helper_classes import SequentialCohortEnrollment
from flourish_caregiver.helper_classes.schedule_dict import child_schedule_dict, caregiver_schedule_dict
from flourish_child.models import Appointment as ChildAppointment
from edc_appointment.models import Appointment
from edc_appointment.constants import NEW_APPT
from edc_visit_schedule import site_visit_schedules
from edc_base.utils import get_utcnow, age, relativedelta
from tqdm import tqdm

In [2]:
child_identifiers = CaregiverChildConsent.objects.exclude(
    preg_enroll=True).values_list('subject_identifier', flat=True).distinct()
child_identifiers = list(set(child_identifiers))

In [ ]:
# Order pids by aged out
child_identifiers.sort()
upper_limit = {'cohort_a': 5, 'cohort_b': 10, 'cohort_a_sec': 5, 'cohort_b_sec': 10, }
ordered_child_idx = []
for idx in tqdm(child_identifiers):
    earliest_consent = CaregiverChildConsent.objects.filter(subject_identifier=idx).earliest('consent_datetime')
    cohort = earliest_consent.cohort
    child_age = age(earliest_consent.child_dob, get_utcnow().date())
    child_age = round(child_age.years + (child_age.months/12) + (child_age.days/365.25), 2)
    age_limit = upper_limit.get(cohort)
    if age_limit and child_age > age_limit:
        ordered_child_idx.append(idx)
        child_identifiers.remove(idx)

In [ ]:
child_identifiers = ordered_child_idx + child_identifiers
len(child_identifiers)
child_identifiers

In [3]:
# Create cohort instance for enrolment cohort
count = 0
for idx in tqdm(child_identifiers):
    childconsent_obj = CaregiverChildConsent.objects.filter(subject_identifier=idx).earliest('consent_datetime')
    obj, created = Cohort.objects.get_or_create(
        subject_identifier=childconsent_obj.subject_identifier,
        name=childconsent_obj.cohort,
        defaults={'assign_datetime': get_utcnow(), 'enrollment_cohort': True})
    if created:
        count += 1
print(f'Created {count} cohort instance(s)')

100%|██████████| 989/989 [00:43<00:00, 22.87it/s]

Created 0 cohort instance(s)


In [3]:
# Current age cohort recalculation
# import math

# records = []
# age_limits = {'cohort_a': 5,
#               'cohort_b': 10, }
# for idx in tqdm(child_identifiers):
#     childconsent_obj = CaregiverChildConsent.objects.filter(subject_identifier=idx).earliest('consent_datetime')
#     if not childconsent_obj.is_preg:
#         enrol_cohort = childconsent_obj.cohort
#         cohort = cohort_assigned(
#             childconsent_obj.study_child_identifier,
#             childconsent_obj.child_dob,
#             get_utcnow().date())

#         child_age = age(childconsent_obj.child_dob, get_utcnow().date())
#         child_age = round(child_age.years + (child_age.months/12) + (child_age.days/365.25), 2)
#         caregiver_subject_identifier = childconsent_obj.subject_consent.subject_identifier
#         if (cohort and enrol_cohort != cohort and cohort != f'{enrol_cohort}_sec'):
#             limit_age = age_limits.get(enrol_cohort, 0)
#             difference = (child_age - limit_age) * 365.25
#             date_aged = get_utcnow().date() - relativedelta(days=difference)

#             records.append({'subject_identifier': idx,
#                             'caregiver_subject_identifier': caregiver_subject_identifier,
#                             'child_count': childconsent_obj.caregiver_visit_count,
#                             'child_dob': childconsent_obj.child_dob,
#                             'enrolment_cohort': enrol_cohort,
#                             'current_age': child_age,
#                             'current_cohort': cohort,
#                             'date_aged_up': date_aged})

100%|██████████| 989/989 [02:34<00:00,  6.42it/s]


In [ ]:
# no_followup = []
# followup = []
# for record in records:
#     enrol_cohort = record.get('enrolment_cohort')
#     if 'sec' in enrol_cohort:
#         no_followup.append(record)
#         continue
#     model = child_schedule_dict[enrol_cohort]['followup']['onschedule_model']
#     model_cls = django_apps.get_model(model)
#     try:
#         model_cls.objects.get(subject_identifier=record.get('subject_identifier'))
#     except model_cls.DoesNotExist:
#         no_followup.append(record)
#     else:
#         followup.append(record)

In [ ]:
# import csv
# keys = no_followup[0].keys()
# with open('main_to_sec.csv', 'w', newline='') as output_file:
#     dict_writer = csv.DictWriter(output_file, keys)
#     dict_writer.writeheader()
#     dict_writer.writerows(no_followup)

In [11]:
def take_off_schedule(onschedule_model_obj):
    subject_identifier = onschedule_model_obj.subject_identifier
    schedule_name = onschedule_model_obj.schedule_name
    onschedule_model = onschedule_model_obj.onschedule_model

    _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
        onschedule_model=onschedule_model, name=schedule_name)

    if schedule.is_onschedule(
        subject_identifier=subject_identifier, report_datetime=get_utcnow()):
        schedule.take_off_schedule(
            subject_identifier=subject_identifier,
            schedule_name=schedule_name)


In [12]:
def delete_completed_appointments(appointment_model_cls, subject_identifier,
                                  schedule_name ):
    complete_appts = appointment_model_cls.objects.filter(
        Q(schedule_name__icontains='quart') | Q(schedule_name__icontains='qt'),
        subject_identifier=subject_identifier, ).exclude(
            appt_status=NEW_APPT).values_list('visit_code', flat=True).distinct()

    new_appts = appointment_model_cls.objects.filter(
        subject_identifier=subject_identifier,
        schedule_name=schedule_name,
        visit_code__in=complete_appts)
    if new_appts.exists():
        new_appts.delete()

In [13]:
def update_onschedule_model(subject_identifier, child_subject_identifier,
                            onschedule_model, schedule_name, ):

    onschedule_model_cls = django_apps.get_model(onschedule_model)
    try:
        onschedule_model_cls.objects.get(
            subject_identifier=subject_identifier,
            schedule_name=schedule_name,
            child_subject_identifier=child_subject_identifier)
    except onschedule_model_cls.DoesNotExist:
        try:
            onschedule_obj = onschedule_model_cls.objects.get(
                subject_identifier=subject_identifier,
                schedule_name=schedule_name)
        except schedule.onschedule_model_cls.DoesNotExist:
            pass
        else:
            onschedule_obj.child_subject_identifier = child_subject_identifier
            onschedule_obj.save()

In [14]:
def put_on_schedule(onschedule_model, schedule_name,
                    subject_identifier, base_appt_datetime=None, ):
    _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
        onschedule_model=onschedule_model,
        name=schedule_name)

    schedule.put_on_schedule(
        subject_identifier=subject_identifier,
        onschedule_datetime=get_utcnow(),
        base_appt_datetime=base_appt_datetime,
        schedule_name=schedule_name)

In [15]:
def put_onschedule(cohort, schedule_type, schedule_dict, onschedule_datetime,
                   subject_identifier, appointment_cls, is_caregiver=False, child_count=None):
    onschedule_model = schedule_dict[cohort][schedule_type]['onschedule_model']
    if is_caregiver:
        schedule_name = schedule_dict[cohort][schedule_type][child_count]
    else:
        schedule_name = schedule_dict[cohort][schedule_type]['name']

    _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
            onschedule_model=onschedule_model,
            name=schedule_name)

    if not schedule.is_onschedule(
        subject_identifier=subject_identifier, report_datetime=onschedule_datetime):

        put_on_schedule(onschedule_model=onschedule_model,
                        schedule_name=schedule_name,
                        base_appt_datetime=onschedule_datetime,
                        subject_identifier=subject_identifier)

        delete_completed_appointments(
                appointment_model_cls=appointment_cls,
                subject_identifier=subject_identifier,
                schedule_name=schedule_name) 
            
    return onschedule_model, schedule_name


In [16]:
def get_onschedule_obj(subject_identifier, cohort):
    cohort = f'{cohort}_' if 'sec' not in cohort else cohort
    try:
        schedule_obj = SubjectScheduleHistory.objects.filter(
            subject_identifier=subject_identifier,
            schedule_name__icontains=cohort.replace('cohort_', '')).filter(
            Q(schedule_name__icontains='qt') | Q(schedule_name__icontains='quart')).latest('onschedule_datetime')
    except SubjectScheduleHistory.DoesNotExist:
        raise
    else:
        return schedule_obj

In [17]:
def get_schedule_type(onschedule_obj, current_cohort):
    schedule_name = onschedule_obj.schedule_name
    if 'fu' in schedule_name and '_sec' not in current_cohort:
        return 'followup_quarterly'
    else:
        return 'quarterly'

In [18]:
cohort_ages = {'cohort_b': 7, 'cohort_c': 12}
def enrol_fu_schedule(child_age, cohort, schedule_dict, subject_identifier, child_sidx,
                      appointment_cls, is_caregiver=False, child_count=None):
    onschedule_datetime = None
    age_fu = cohort_ages.get(cohort, child_age)
    if child_age < age_fu:
        age_diff = round(age_fu - child_age, 2)
        age_in_months = round(age_diff * 12)
        onschedule_datetime = get_utcnow() + relativedelta(months=age_in_months)
    else:
        onschedule_datetime = get_utcnow() + relativedelta(months=6)

    if onschedule_datetime:
        onschedule_model, schedule_name = put_onschedule(
            cohort, 'followup', schedule_dict, onschedule_datetime,
            subject_identifier, appointment_cls, is_caregiver=is_caregiver, child_count=str(child_count))

        if is_caregiver:
            update_onschedule_model(subject_identifier, child_sidx,
                                    onschedule_model, schedule_name, )

In [19]:
# child_identifiers=['B142-040990096-5-25', 'B142-040990096-5-35']

In [21]:
# from cacheops import invalidate_model
"""
    Take participant offschedule for enrolment cohort and put onschedule for subsequent cohort schedule
    1. Create a cohort instance for `current cohort`,
        NOTE: If cohort instance already exists assume already put onschedule
    2. Take offschedule for `enrolment cohort`
    3. Put onschedule for `current cohort`
    4. Make sure to align appointments for `current cohort` schedule with enrolment
"""
for idx in tqdm(child_identifiers):
    childconsent_obj = CaregiverChildConsent.objects.filter(subject_identifier=idx).earliest('consent_datetime')
    if childconsent_obj.is_preg:
        continue
    enrol_cohort = childconsent_obj.cohort
    current_cohort = cohort_assigned(
        childconsent_obj.study_child_identifier,
        childconsent_obj.child_dob,
        get_utcnow().date())

    caregiver_subject_identifier = childconsent_obj.subject_consent.subject_identifier
    if (current_cohort and enrol_cohort != current_cohort and current_cohort != f'{enrol_cohort}_sec'):
        if idx in ['B142-040990120-3-10', 'B142-040990263-1-10', 'B142-040990341-5-10',
                   'B142-040990466-0-10', 'B142-040990612-9-60', 'B142-040990666-5-60',
                   'B142-040990755-6-10', 'B142-040990943-8-10', 'C142-040990599-8-10',
                   'B142-040990528-7-60']:
            continue
        if not bool(SubjectScheduleHistory.objects.onschedules(subject_identifier=idx)):
            continue

        print('start', idx)
        child_age = age(childconsent_obj.child_dob, get_utcnow().date())
        child_age = (child_age.years + (child_age.months/12))

        obj, created = Cohort.objects.get_or_create(
            subject_identifier=idx,
            name=current_cohort,
            defaults={'assign_datetime': get_utcnow(), 'enrollment_cohort': False})

        # Take child offschedule
        onschedule_obj = get_onschedule_obj(idx, enrol_cohort)
        take_off_schedule(onschedule_obj)

        # Put child onschedule
        schedule_type = get_schedule_type(onschedule_obj, current_cohort)
        put_onschedule(current_cohort, schedule_type, child_schedule_dict,
                       onschedule_obj.onschedule_datetime,
                       idx, ChildAppointment, )

        # Put child onschedule for followup
        if '_sec' not in current_cohort:
            enrol_fu_schedule(
                child_age, current_cohort, child_schedule_dict, idx, idx,
                ChildAppointment, is_caregiver=False, child_count=None)

        # Take caregiver offschedule
        onschedule_obj = get_onschedule_obj(caregiver_subject_identifier, enrol_cohort)
        take_off_schedule(onschedule_obj)

        # Put caregiver onschedule
        schedule_type = get_schedule_type(onschedule_obj, current_cohort)
        child_count = childconsent_obj.caregiver_visit_count
        onschedule_model, schedule_name = put_onschedule(
            current_cohort, schedule_type, caregiver_schedule_dict,
            onschedule_obj.onschedule_datetime,
            caregiver_subject_identifier, Appointment, is_caregiver=True, child_count=str(child_count))

        # Update child subject identifier
        update_onschedule_model(caregiver_subject_identifier, idx,
                                onschedule_model, schedule_name, )

        # Put caregiver onschedule for followup
        if '_sec' not in current_cohort:
            enrol_fu_schedule(
                child_age, current_cohort, caregiver_schedule_dict, caregiver_subject_identifier, idx,
                Appointment, is_caregiver=True, child_count=str(child_count))

        print('done', idx)
#         invalidate_model(Cohort)
#         invalidate_all()

  0%|          | 0/2 [00:00<?, ?it/s]

start B142-040990096-5-25


 50%|█████     | 1/2 [00:10<00:10, 10.45s/it]

done B142-040990096-5-25
start B142-040990096-5-35


100%|██████████| 2/2 [00:19<00:00,  9.55s/it]

done B142-040990096-5-35


In [22]:
cohorts = Cohort.objects.filter(subject_identifier__in=child_identifiers)
for cohort in cohorts:
    cohort.save()

In [ ]:
# Data fix to complete the caregiver visit count on the child consent.
child_consents =  CaregiverChildConsent.objects.values_list('subject_identifier', flat=True)


caregiver_models = dir(django_apps.get_app_config('flourish_caregiver').models_module.onschedule)
caregiver_models = [model for model in caregiver_models if model.lower().startswith('onschedulecohort')]

for idx in twins:
    for onschedule in caregiver_models:
        onschedule = onschedule.lower()
        onschedule_cls = django_apps.get_model(f'flourish_caregiver.{onschedule}')

        reg_subject = RegisteredSubject.objects.get(subject_identifier=idx)
        onschedule_obj = onschedule_cls.objects.filter(subject_identifier=reg_subject.relative_identifier).first()
        if not onschedule_obj:
            continue
        schedule_name = onschedule_obj.schedule_name[:-1]
        consents = CaregiverChildConsent.objects.filter(subject_identifier=idx)
        for index in range(len(schedule_name)):
            if schedule_name[index].isdigit():
                consents.update(caregiver_visit_count=int(schedule_name[index]))
        break

twins = CaregiverChildConsent.objects.filter(
    Q(subject_identifier__endswith='-25') | Q(subject_identifier__endswith='-35')).values_list('subject_identifier', flat=True)
twins = list(set(twins))
# Use registered subject and relative identifier to update them